In [46]:
from bs4 import BeautifulSoup as bs
import grequests
import requests
import pandas as pd


imo = requests.get('https://www.imovirtual.com/arrendar/apartamento/') # request http
raw_html = imo.text # convert webpage code into raw text
soup = bs(raw_html) # soupify page text

# find number of pages
for i in soup.find_all('ul', class_='pager'):
    pages = int(i.text.split()[-1])
    
# initialize url list with first page for subsequent appending 
urls = ['https://www.imovirtual.com/arrendar/apartamento/'] 

# append further pages links
for page in range(2,pages+1):
    urls.append('https://www.imovirtual.com/arrendar/apartamento/?page=' + str(page))
    
# initialize lists for house prices, types, locations and sizes
prices = []
types = []
location = []
sizes = [] 

for u in urls:
    imo = requests.get(u) # request http
    raw_html = imo.text # convert webpage code into raw text
    soup = bs(raw_html) # soupify page text

    # Find the desired data: price, location (concelho), typology and size (m2)

    # price
    for price in soup.find_all('li', class_="offer-item-price"): 
        prices.append(price.string.split('€')[0].replace(' ','').strip())

    #type 
    for ty in soup.find_all('li', class_="offer-item-rooms hidden-xs"):
        types.append(ty.string)

    #location
    for loc in soup.find_all('p', class_="text-nowrap"):
        location.append(loc.text.split('Apartamento para arrendar: ')[1])

    #size (m2)
    for size in soup.find_all('li', class_="hidden-xs offer-item-area"):
        sizes.append(size.next.split(' ')[0])
        

        
all_houses = [] # list where all houses will be
all_houses.append([prices, types, location, sizes])
columns = ['price','type','location','size'] # columns names for the dataframe
all_data = pd.DataFrame(all_houses[0]) # create datafame with the list created
all_data = all_data.transpose() # transpose dataframe
all_data.columns = columns # rename dataframe columns

# replace house types with numeric values       
type_dict = {'T2':2, 'T3':3, 'T1':1, 'T4':4, 'T0':0, 'T5':5, 'T6':6, 'T8':8, 'T7':7, 'T10 ou superior':10, 'T9':9}
all_data['type'] = all_data['type'].replace(type_dict)

all_data

,price,type,location,size
0,1000,2.0,"Rua António Ferreira Fiandor - Laborim, Mafamu...",98
1,1350,3.0,"Santa Marinha e São Pedro da Afurada, Vila Nov...",127
2,1200,4.0,"São Domingos de Benfica, Lisboa",146
3,700,1.0,"Odivelas, Lisboa",75
4,950,0.0,"Paranhos, Porto",41
...,...,...,...,...
11972,750,2.0,"Buarcos e São Julião, Figueira da Foz, Coimbra",60
11973,700,3.0,"Arroios, Lisboa",125
11974,650,2.0,"Cedofeita, Santo Ildefonso, Sé, Miragaia, São ...",83
11975,975,2.0,"São Victor, Braga",85
